In [ ]:
###########
# INSTALL LIBRARIES
##################

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing import image_dataset_from_directory
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


2024-06-27 10:38:01.686359: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-27 10:38:01.686464: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-27 10:38:01.797940: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:


def load_dataset(data_path):
    """
    Loads an image training dataset from the specified directory.

    Parameters:
    - data_path (str): Path to the directory containing the image files.

    Returns:
    - train_dataset: A tf.data.Dataset object containing the loaded images along with their labels.
    """
    # Load the training dataset from the specified directory
    train_dataset = image_dataset_from_directory(
        data_path,
        labels='inferred',  # Infer labels from the directory structure
        label_mode='int',   # Labels are returned as integers (for classification tasks)
        image_size=(64, 64),  # Resize images to 64x64 (corrected from the mismatched comment)
        color_mode='rgb',  # Use RGB color mode
        batch_size=32,  # Number of samples per batch
        shuffle=True,  # Shuffle the dataset
        seed=42,  # Seed used for shuffling and transformations
        validation_split=0.0,  # No splitting; all data used as training set
        subset=None,  # Subset of data to use ('training' or 'validation'), not used here
    )

    # Initialize lists to store batched data
    X_train = []
    y_train = []

    # Iterate through the dataset and collect all data into lists
    for images, labels in train_dataset:
        X_train.append(images.numpy())
        y_train.append(labels.numpy())

    # Convert lists to a single numpy array
    X_train = np.concatenate(X_train)
    y_train = np.concatenate(y_train)

    # Normalize images to range [-1, 1]
    X_train = (X_train / 127.5) - 1

    return X_train, y_train

# Remember to adjust the normalization and image size as necessary based on your specific use case.


In [4]:

#################
     #PATHS
#################
# Define paths used in the project:
input_folder = Path('/kaggle/input/')  # Set the path for the input folder where data files are stored.
output_folder = Path('/kaggle/working/')  # Set the path for the output folder where results and outputs will be saved.
# Define the path to the image classification dataset within the input folder.
data_path = input_folder / 'char-class/images-classification'
# Load the dataset from the specified directory using a function assumed to handle dataset loading.
# The function 'load_dataset' would typically preprocess and ready the dataset for use in training or testing.
dataset = load_dataset(data_path)



In [ ]:
dataset[0].shape

In [ ]:
dataset[1].shape

In [ ]:
def show_image_with_label(X, y, index):
    # Retrieve the image and label from the dataset using the provided index.
    image = X[index]
    label = y[index]

    # Reverse normalization to display the image correctly if it was normalized between -1 and 1.
    image = (image + 1) / 2.0

    # Use matplotlib to display the image.
    plt.imshow(image.squeeze())  # Squeeze is used to remove single-dimensional entries from the shape of the image.
    plt.title(f'Label: {label} | Dimensions: {image.shape}')  # Set the title of the plot to show the label and image dimensions.
    plt.axis('off')  # Turn off the axis.
    plt.show()  # Display the plot.

    # Print additional information about the image.
    print(f"Image array (index {index}):\n", image)
    print(f"Label: {label}")
    print(f"Dimensions: {image.shape}")

# Load the training dataset (assuming 'dataset' is a tuple of (X_train, y_train)).
X_train, y_train = dataset

# Display the first image and its label using the function.
show_image_with_label(X_train, y_train, 3988)


In [ ]:

def plot_results(images, n_cols=None):
    """
    Displays a list of images in a grid format.

    Parameters:
    - images (list of np.array): A list of images (as numpy arrays) normalized between [-1, 1].
    - n_cols (int, optional): The number of columns in the grid display. If not specified, the number
      of columns will be equal to the number of images, resulting in a single row.

    This function converts images from a [-1, 1] normalization to [0, 1] by adding 1 and dividing by 2,
    which is suitable for displaying with imshow that expects values between [0, 1].
    """

    # Determine the number of columns and rows for the display
    n_cols = n_cols or len(images)  # If n_cols is not specified, use the total number of images
    n_rows = (len(images) - 1) // n_cols + 1  # Calculate the necessary number of rows

    # Normalize the images from [-1, 1] to [0, 1]
    images = (images + 1.0) / 2.0  # Prepare images for correct display

    # Configure the figure size based on the number of columns and rows
    plt.figure(figsize=(2 * n_cols, 2 * n_rows))  # Adjust figure size for better visibility

    # Display each image from the list
    for index, image in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)  # Create a subplot for each image
        plt.imshow(image)  # Display the image
        plt.axis("off")  # Hide the axes to focus only on the images

    plt.show()  # Show the final result


In [ ]:

def get_dataset_samples(dataset, n_samples):
    """
    Randomly selects a subset of samples from a dataset.

    Parameters:
    - dataset (tuple): A tuple containing two elements: the images and the labels.
    - n_samples (int): The number of samples to select.

    Returns:
    - A tuple ([X, labels], y) where 'X' are the selected images, 'labels' are the corresponding labels,
      and 'y' is an array of ones (often used for specific tasks).

    The function assumes that 'dataset' is a tuple where the first element is an array of images and the
    second element is an array of labels. It returns a specified subset of these images and labels.
    """

    # Decompose the dataset into images and labels
    images, labels = dataset

    # Generate random indices for the selection of samples
    ix = np.random.randint(0, images.shape[0], n_samples)

    # Select the images and corresponding labels
    X, labels = images[ix], labels[ix]

    # Create an array of ones for potential use (depending on the usage context)
    y = np.ones((n_samples, 1))

    return [X, labels], y


In [ ]:

def generate_noise(noise_size, n_samples, n_classes=10):
    """
    Generates a noise dataset with corresponding random class labels.

    Parameters:
    - noise_size (int): The size of the noise vector for each sample.
    - n_samples (int): The number of samples to generate.
    - n_classes (int, optional): The number of classes to generate labels for, defaults to 10.

    Returns:
    - A tuple ([z_input, labels]) where 'z_input' is a matrix of noise vectors and 'labels' is an array of random class labels.

    Cette fonction génère un ensemble de données de bruit avec des étiquettes de classe aléatoires correspondantes.
    """

    # Generate random Gaussian noise
    x_input = np.random.randn(noise_size * n_samples)  # Generate Gaussian noise
    z_input = x_input.reshape(n_samples, noise_size)  # Reshape to form a matrix

    # Generate random labels for each sample
    labels = np.random.randint(0, n_classes, n_samples)  # Random integer labels from 0 to n_classes-1

    return [z_input, labels]

# Example of using the function:
noise_size = 100  # Size of the noise vector
n_samples = 64   # Number of samples to generate
n_classes = 10   # Number of classes

noise_data = generate_noise(noise_size, n_samples, n_classes)
print(noise_data[0].shape)  # Prints the shape of the noise matrix
print(noise_data[1].shape)  # Prints the shape of the labels array


In [ ]:




def generate_fake_samples(generator, latent_dim, n_samples):
    """
    Generates fake samples using a generator model from a GAN.

    Parameters:
    - generator (Model): The generator model from a GAN setup that takes noise and labels as input.
    - latent_dim (int): The size of the latent space (dimensionality of the noise vector).
    - n_samples (int): The number of fake samples to generate.

    Returns:
    - A tuple containing the generated fake samples and their corresponding labels:
      ([images, labels_input], y) where 'images' are the generated images, 'labels_input' are the random class labels,
      and 'y' is an array of zeros indicating the samples are fake.
    
    Utilise un modèle générateur pour transformer des entrées de bruit en données synthétiques (images),
    et génère également des étiquettes indiquant que ces échantillons sont 'faux'.
    """

    # Generate noise and labels
    z_input, labels_input = generate_noise(latent_dim, n_samples)  # Use the helper function to generate inputs

    # Generate fake images using the generator model
    images = generator.predict([z_input, labels_input])  # Predict (generate) images using the generator

    # Create an array of zeros to label the generated images as fake
    y = np.zeros((n_samples, 1))  # Labels for the GAN discriminator (0 = fake)

    return [images, labels_input], y



In [ ]:
def define_generator(latent_dim, n_classes=24):
    # Input layer for the label, one-hot encoded.
    in_label = keras.layers.Input(shape=(1,))
    # Embedding layer to transform the label into a dense representation.
    li = keras.layers.Embedding(n_classes, 50)(in_label)
    # Define the number of nodes to match the image structure expected later.
    n_nodes = 7 * 7
    # Dense layer to expand the label embedding.
    li = keras.layers.Dense(n_nodes)(li)
    # Reshape the output to be compatible with the image structure.
    li = keras.layers.Reshape((7, 7, 1))(li)
    
    # Input layer for generating images from the latent space.
    in_lat = keras.layers.Input(shape=(latent_dim,))
    # Define the number of nodes for the generator's first layer.
    n_nodes = 128 * 7 * 7
    # Dense layer to process the input latent vector.
    gen = keras.layers.Dense(n_nodes)(in_lat)
    # LeakyReLU for non-linearity while allowing small gradients when the unit is not active.
    gen = keras.layers.LeakyReLU(alpha=0.2)(gen)
    # Reshape to start forming the image.
    gen = keras.layers.Reshape((7, 7, 128))(gen)

    # Concatenate the label conditioning and the latent input into a single tensor.
    merge = keras.layers.Concatenate()([gen, li])
    # First transposed convolution layer to upscale the image.
    gen = keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',
                                       activation=keras.layers.LeakyReLU(alpha=0.2))(merge)
    # Batch normalization to stabilize learning.
    gen = keras.layers.BatchNormalization()(gen)
    # Second transposed convolution layer to continue upscaling.
    gen = keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',
                                       activation=keras.layers.LeakyReLU(alpha=0.2))(gen)
    # Batch normalization layer.
    gen = keras.layers.BatchNormalization()(gen)
    # Output convolution layer that converts the image tensor to the final image shape with 3 channels.
    out_layer = keras.layers.Conv2D(3, (3,3), activation='tanh', padding='same')(gen)

    # Create the model object that takes two inputs and outputs the generated image.
    model = keras.Model([in_lat, in_label], out_layer)
    return model


In [ ]:
def define_generator(noise_dim=100, n_classes=10):
    # Inputs: a label and a noise vector. The label is embedded and reshaped to match the dimensionality of the transformation process.
    label_input = keras.layers.Input(shape=(1,), dtype='int32')
    label_embedding = keras.layers.Embedding(n_classes, noise_dim)(label_input)
    label_reshape = keras.layers.Reshape((8, 8, 1))(keras.layers.Dense(8 * 8)(label_embedding))

    # Process the noise vector to generate features for the generator.
    noise_input = keras.layers.Input(shape=(noise_dim,))
    noise_reshape = keras.layers.Reshape((8, 8, 128))(keras.layers.Dense(8 * 8 * 128)(noise_input))

    # Merge the noise-derived features and label-derived features.
    merged = keras.layers.Concatenate()([noise_reshape, label_reshape])

    # Convolutional layers to upsample the combined feature map to the target image size.
    gen = keras.layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merged)
    gen = keras.layers.LeakyReLU(alpha=0.2)(gen)  # Repeatedly upsample and apply LeakyReLU.

    # Final convolutional layer to produce the output image with 3 color channels (RGB), using the 'tanh' activation to scale the pixel values between -1 and 1.
    out_layer = keras.layers.Conv2D(3, (3,3), activation='tanh', padding='same')(gen)

    # Assemble the model with noise and label inputs.
    model = keras.Model([noise_input, label_input], out_layer)
    return model



In [ ]:
noise_size = 100

generator = define_generator(noise_size)

generator.summary()

In [ ]:
def define_discriminator(in_shape=(64, 64, 3), n_classes=24):
    # Input for the label, embedded to a dense representation suitable for merging with image data.
    in_label = keras.layers.Input(shape=(1,))
    li = keras.layers.Embedding(n_classes, 50)(in_label)
    n_nodes = in_shape[0] * in_shape[1]
    li = keras.layers.Dense(n_nodes)(li)
    li = keras.layers.Reshape((in_shape[0], in_shape[1], 1))(li)

    # Input for the image.
    in_image = keras.layers.Input(shape=in_shape)

    # Merge the image and label inputs to form the combined feature map.
    merge = keras.layers.Concatenate()([in_image, li])

    # Convolutional layers to downsample the input and extract features, with dropout for regularization.
    fe = keras.layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same', activation=keras.layers.LeakyReLU(alpha=0.2))(merge)
    fe = keras.layers.Dropout(0.4)(fe)
    fe = keras.layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same', activation=keras.layers.LeakyReLU(alpha=0.2))(fe)
    fe = keras.layers.Dropout(0.4)(fe)

    # Flatten the feature map and use a dense layer to output a single value indicating real or fake.
    fe = keras.layers.Flatten()(fe)
    out_layer = keras.layers.Dense(1, activation='sigmoid')(fe)

    # Create the model with image and label as inputs.
    model = keras.Model([in_image, in_label], out_layer)

    # Compile the model with the Adam optimizer and binary crossentropy loss, also tracking accuracy.
    opt = keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model


In [ ]:
discriminator = define_discriminator()
discriminator.summary()

In [ ]:
def define_gan(generator, discriminator):
    # Ensure the discriminator's weights are not updated during the combined model's training.
    discriminator.trainable = False

    # Retrieve the input layers from the generator model.
    gen_noise, gen_label = generator.input

    # Get the output from the generator model.
    gen_output = generator.output

    # Feed the generator's output along with the label input into the discriminator.
    gan_output = discriminator([gen_output, gen_label])

    # Define the GAN model with noise and label inputs leading to the discriminator's output.
    model = keras.Model([gen_noise, gen_label], gan_output)

    # Configure the model using the Adam optimizer and compile it with binary cross-entropy loss.
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

    return model


In [ ]:
GAN = define_gan(generator, discriminator)
GAN.summary()

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
def train_gan(generator, discriminator, GAN, dataset, noise_size=100, n_epochs=30, n_batch=16):
    # Calculate the number of steps per epoch based on batch size and dataset size.
    steps = int(dataset[0].shape[0] / n_batch)
    # Define half batch size for training the discriminator on real and fake data separately.
    half_batch = int(n_batch / 2)

    # Loop through each epoch.
    for e in range(n_epochs):
        print("EPOCH", e)
        # Loop through each step within the epoch.
        for s in range(steps):
            # Get a subset of real samples from the dataset.
            [X_real, labels_real], y_real = get_dataset_samples(dataset, half_batch)
            # Train the discriminator on real data.
            d_loss1, _ = discriminator.train_on_batch([X_real, labels_real], y_real)

            # Generate a set of fake samples.
            [X_fake, labels], y_fake = generate_fake_samples(generator, noise_size, half_batch)
            # Train the discriminator on fake data.
            d_loss2, _ = discriminator.train_on_batch([X_fake, labels], y_fake)

            # Generate noise and corresponding labels for the generator input.
            [z_input, labels_input] = generate_noise(noise_size, n_batch)
            # All labels for GAN training are set to one (as GAN's output should be seen as 'real').
            y_gan = np.ones((n_batch, 1))
            # Train the GAN by updating the generator via the discriminator’s error.
            g_loss = GAN.train_on_batch([z_input, labels_input], y_gan)
            # Optional: Uncomment to print losses for each step.
            # print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' % (e+1, s+1, steps, d_loss1, d_loss2, g_loss))
        
        # Display generated images from the last batch of the epoch.
        plot_results(X_fake, 8)  
    
    # Save the generator model at the end of training.
    generator.save('cgan_generator.h5')


In [ ]:
dataset = load_dataset(data_path)


In [ ]:
# Train the GAN model with the following parameters:
# - generator: the generator model to produce fake samples.
# - discriminator: the discriminator model to evaluate real and fake samples.
# - GAN: the combined model that connects the generator and discriminator.
# - dataset: the dataset used for training, containing real samples and their labels.
# - noise_size: the size of the noise vector used by the generator to create samples.
# - n_epochs: the number of epochs for training, set to 1 in this call.
# - n_batch: the batch size for training, set to 16.

train_gan(generator, discriminator, GAN, dataset, noise_size, n_epochs=1, n_batch=16)


In [ ]:
# Load the pre-trained GAN generator model from the specified file.
model = keras.models.load_model('cgan_generator.h5')

# Generate noise and labels for creating new samples.
# 'noise_size' specifies the dimensionality of the noise vector.
# 'generate_noise' creates 20 noise vectors and corresponding labels.
latent_points, labels = generate_noise(noise_size, 20)

# Set all labels to 0 for the generated samples.
labels = np.ones(20) * 0

# Use the generator model to predict (generate) new images based on the noise and labels.
X = model.predict([latent_points, labels])

# Plot and display the generated images in a grid format.
plot_results(X, 10)


In [ ]:
# Generate noise and labels for prediction
latent_points, labels = generate_noise(100, 20)  # Use noise dimension 100

# Assuming you want to generate images with label 0
labels = np.ones(latent_points.shape[0]) * 1  # Set labels for all generated noise points to 0

# Generate images
X = model.predict([latent_points, labels])

# Plot the generated images
plot_results(X, int(np.sqrt(X.shape[0])))


